## **Import Relevant Packages**

In [ ]:
# Install Package

pip install feature_engine

In [ ]:
# Import Packages

import os
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from IPython.display import Image
from sklearn.metrics import confusion_matrix
from feature_engine.encoding import OrdinalEncoder
from sklearn.model_selection import train_test_split

## **1. PreProcessing Data**

### 1. Importing Dataset

In [ ]:
# Import labels data (id, label)

data = pd.read_csv("../input/data-augment/all_labels.csv")
data.head()

### 2. Change ID to File Path

In [ ]:
# Change id to file path

file_path = []
for x in data["path"]:
    file_path.append(f"../input/data-augment/all/{x}")

data["path"] = file_path
data = data[["path", "label"]]
data.head()

### 3. Turn Labels (y) into Number (Tensors)

In [ ]:
# Prepare Label
labels_train = np.array(data['label'])

# Unique Label/Breed
unique_labels = np.unique(labels_train)
unique_labels

In [ ]:
# Turn each label into an array of booleans
boolean_labels_train = [label == unique_labels for label in labels_train]

### 4. Split Dataset into Train & Validation

In [ ]:
# TRAIN & VALIDATION
X_train = data["path"]
y_train = boolean_labels_train

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [ ]:
# Cek total training and validation set
print("Train Set:", len(X_train))
print("Validation Set:", len(X_val))

### 5. Turn Images (X) into Number (Tensors)

In [ ]:
# Train set
train_set = tf.data.Dataset.from_tensor_slices((tf.constant(X_train),
                                                tf.constant(y_train)))
# Validation Set
val_set = tf.data.Dataset.from_tensor_slices((tf.constant(X_val),
                                              tf.constant(y_val)))

### 6. Shuffling Train Set

In [ ]:
# Shuffle train set
BUFFER_SIZE = len(train_set)
train_set = train_set.shuffle(buffer_size=BUFFER_SIZE)

### 7. Function for Preprocessing Data

In [ ]:
# Function for process image

def process_image(image_path):
    # Read an image
    image = tf.io.read_file(image_path)
    
    # Turn jpeg to numbers (isinya probability color 0-255)
    image = tf.image.decode_jpeg(image, channels=3)

    # Convert the color channel values from 0-255 to 0-1
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    image /= 255
    
    # Resize image to our desired shape (height,width)
    image = tf.image.resize(image, size=[470,372])

    return image

In [ ]:
# Function to get tuple of the dataset

def get_tuple(image_path, label):
    # Pre-process dulu sama function yg tadi
    image = process_image(image_path)
    
    # Return tuple
    return image, label

In [ ]:
# Apply function to train set
train_set = train_set.map(get_tuple)

# Apply function to validation set
val_set = val_set.map(get_tuple)

### 8. Turn Data into Batches

In [ ]:
BATCH_SIZE = 1 #standard

# train set
train_set = train_set.batch(BATCH_SIZE)

# validation set
val_set = val_set.batch(BATCH_SIZE)

In [ ]:
# Check element spec
train_set.element_spec, val_set.element_spec   # batch, height, width, channel

## **2. Building The ANN Model**


### 1. Create Model

In [ ]:
# Setup the input shape
INPUT_SHAPE = [None, 470, 372, 3] #batch, height, width, colour channels

# Setup the output shape
OUTPUT_SHAPE = len(unique_labels)
print(OUTPUT_SHAPE)

In [ ]:
# Initialize model

tf.keras.applications.mobilenet_v2.trainable = False
model = tf.keras.Sequential([
        tf.keras.applications.mobilenet_v2.MobileNetV2(input_shape = (470,372,3), weights = None, include_top = False),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(60),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10),
        tf.keras.layers.Dense(units=OUTPUT_SHAPE, activation = "softmax")],
        name = "MobileNetV2_Categorical_Classification")

model.compile(optimizer = 'adam',
                  loss = "categorical_crossentropy",
                  metrics = ["accuracy"])

In [ ]:
# view the structure of the model
model.summary()

### 3. Train Model

In [ ]:
history = model.fit(train_set,
                    epochs = 3,
                    validation_data = val_set,
                    verbose = 2)

## **3. Making Predictions & Evaluating Model**


In [ ]:
# Make predictions on validation set
predictions = model.predict(val_set)

In [ ]:
# Turn prediction probabilities into prediction labels
def get_pred_label(prediction_probabilities):
    return unique_labels[np.argmax(prediction_probabilities)]

# Example
pred_label = get_pred_label(predictions[2])
pred_label

## **4. Saving & Reloading Trained Model**

In [ ]:
# Create a function that save the model

def save_model(model, suffix=None):
    # Read model directory
    modeldir = "./"
    
    # Format path model
    model_path = f"./{suffix}.h5"

    print(f"Saving model to: {model_path}")
    model.save(model_path)
    return print("Model saved!")

In [ ]:
# Save our model 
save_model(model, suffix="augment-hpc_fix")